### Greedy Algorithms

In [5]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.weight = w
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        res = '<'+self.name+', '+str(self.value)+', '+str(self.weight)+'>'
        return res
    
def value(item):
    return item.getValue()
def weightInverse(item):
    return 1.0 / item.getWeight()
def density(item):
    return item.getValue()/item.getWeight()
def buildItems():
    names = ['clock','painting','radio','vase','book','computer']
    vals = [175,90,20,50,10,200]
    weights = [10,9,4,2,1,20]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i],vals[i],weights[i]))
    return Items
def greedy(Items, maxWeight, keyFcn):
    assert type(Items) == list and maxWeight >= 0
    ItemsCopy = sorted(Items, key=keyFcn, reverse=True)
    result = []
    totalVal = 0.0
    totalWeight = 0.0
    i = 0
    while totalWeight < maxWeight and i < len(Items):
        if(totalWeight+ItemsCopy[i].getWeight()) <= maxWeight:
            result.append(ItemsCopy[i])
            totalWeight += ItemsCopy[i].getWeight()
            totalVal += ItemsCopy[i].getValue()
        i += 1
    return (result,totalVal)
def testGreedy(Items, constraint, getKey):
    taken, val = greedy(Items, constraint, getKey)
    print('Total value of items taken = ', str(val))
    for item in taken:
        print(' ',item)

In [6]:
maxWeight = 20
Items = buildItems()
print('Use greedy by value for knapsack of size maxWeight')
testGreedy(Items, maxWeight, value)
print('Use greedy by weight to fill knapsack of size maxWeight')
testGreedy(Items,maxWeight,weightInverse)
print('Use greedy by density for knapsack of size maxWeight')
testGreedy(Items, maxWeight, density)

Use greedy by value for knapsack of size maxWeight
Total value of items taken =  200.0
  <computer, 200, 20>
Use greedy by weight to fill knapsack of size maxWeight
Total value of items taken =  170.0
  <book, 10, 1>
  <vase, 50, 2>
  <radio, 20, 4>
  <painting, 90, 9>
Use greedy by density for knapsack of size maxWeight
Total value of items taken =  255.0
  <vase, 50, 2>
  <clock, 175, 10>
  <book, 10, 1>
  <radio, 20, 4>


In [128]:
#From codereview.stackexchange.com                    
def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(2**len(set_)//2):
        parts = [set(), set()]
        for item in set_:
            parts[i&1].add(item)
            i >>= 1
        for b in partitions(parts[1]):
            yield [parts[0]]+b


# This is a helper function that will fetch all of the available 
# partitions for you to use for your brute force algorithm.
def get_partitions(set_):
    for partition in partitions(set_):
        yield [list(elt) for elt in partition]

### Uncomment the following code  and run this file
### to see what get_partitions does if you want to visualize it:

# for item in (get_partitions(['a','b','c','d'])):
#     print(len(item))

In [155]:
###########################
# 6.00.2x Problem Set 1: Space Cows 

# from ps1_partition import get_partitions
import time

#================================
# Part A: Transporting Space Cows
#================================

def load_cows(filename):
    """
    Read the contents of the given file.  Assumes the file contents contain
    data in the form of comma-separated cow name, weight pairs, and return a
    dictionary containing cow names as keys and corresponding weights as values.

    Parameters:
    filename - the name of the data file as a string

    Returns:
    a dictionary of cow name (string), weight (int) pairs
    """

    cow_dict = dict()

    f = open(filename, 'r')
    
    for line in f:
        line_data = line.split(',')
        cow_dict[line_data[0]] = int(line_data[1])
    return cow_dict


# Problem 1   
def greedy_cow_transport(cows,limit=10):
    """
    Uses a greedy heuristic to determine an allocation of cows that attempts to
    minimize the number of spaceship trips needed to transport all the cows. The
    returned allocation of cows may or may not be optimal.
    The greedy heuristic should follow the following method:

    1. As long as the current trip can fit another cow, add the largest cow that will fit
        to the trip
    2. Once the trip is full, begin a new trip to transport the remaining cows

    Does not mutate the given dictionary of cows.

    Parameters:
    cows - a dictionary of name (string), weight (int) pairs
    limit - weight limit of the spaceship (an int)
    
    Returns:
    A list of lists, with each inner list containing the names of cows
    transported on a particular trip and the overall list containing all the
    trips
    """
    # TODO: Your code here
    trips = []
    sorted_cows = sorted(cows.items(), key=lambda x: x[1], reverse = True) # sorted by weights
    j = 1
    while j <= len(sorted_cows): # still has cows to transport
        cows_per_trip = []
        total_weight = 0
        i = 0
        while total_weight < limit and i < len(sorted_cows):
            if (total_weight + sorted_cows[i][1]) <= limit:
                cows_per_trip.append(sorted_cows[i][0])
                total_weight += sorted_cows[i][1]
                del sorted_cows[i] # delete element in the list, thus no need to change index
            else:
                i += 1 # move to next
        # print(cows_per_trip) use for debug
        trips.append(cows_per_trip)
    return trips
        
# Problem 2
def brute_force_cow_transport(cows,limit=10):
    """
    Finds the allocation of cows that minimizes the number of spaceship trips
    via brute force.  The brute force algorithm should follow the following method:

    1. Enumerate all possible ways that the cows can be divided into separate trips
    2. Select the allocation that minimizes the number of trips without making any trip
        that does not obey the weight limitation
            
    Does not mutate the given dictionary of cows.

    Parameters:
    cows - a dictionary of name (string), weight (int) pairs
    limit - weight limit of the spaceship (an int)
    
    Returns:
    A list of lists, with each inner list containing the names of cows
    transported on a particular trip and the overall list containing all the
    trips
    """
    # TODO: Your code here
    all_possible_trips = [] 

    for item in (get_partitions(cows)): # all possible ways
        all_possible_trips.append(item)
        
    sorted_trips = sorted(all_possible_trips,key=lambda x:len(x)) # sort by length

    Flag = True
    for trip in sorted_trips:
        for transport in trip:
            tot_weights = 0
            for c in transport:
                tot_weights += cows[c]
            if tot_weights > limit:
                Flag = False
                break
            Flag = True
        if Flag:
            return trip
            
        
    
# Problem 3
def compare_cow_transport_algorithms():
    """
    Using the data from ps1_cow_data.txt and the specified weight limit, run your
    greedy_cow_transport and brute_force_cow_transport functions here. Use the
    default weight limits of 10 for both greedy_cow_transport and
    brute_force_cow_transport.
    
    Print out the number of trips returned by each method, and how long each
    method takes to run in seconds.

    Returns:
    Does not return anything.
    """
    # TODO: Your code here
    start = time.time()
    greedy_cow_transport(cows, limit)
    end = time.time()
    print("Greedy algorithm takes " + str(end-start))
    start = time.time()
    brute_force_cow_transport(cows, limit)
    end = time.time()
    print("Brute force algorithm takes " + str(end-start))

In [158]:
cows = load_cows("ps1_cow_data.txt")
limit=100
print(cows)

print(greedy_cow_transport(cows, limit))
print(brute_force_cow_transport(cows, limit))
compare_cow_transport_algorithms()

{'Maggie': 3, 'Herman': 7, 'Betsy': 9, 'Oreo': 6, 'Moo Moo': 3, 'Milkshake': 2, 'Millie': 5, 'Lola': 2, 'Florence': 2, 'Henrietta': 9}
[['Betsy', 'Henrietta', 'Herman', 'Oreo', 'Millie', 'Maggie', 'Moo Moo', 'Milkshake', 'Lola', 'Florence']]
[['Florence', 'Millie', 'Betsy', 'Milkshake', 'Henrietta', 'Moo Moo', 'Herman', 'Maggie', 'Lola', 'Oreo']]
Greedy algorithm takes 0.0
Brute force algorithm takes 0.9977712631225586


### Graph Theory

In [2]:
class Node(object):
    def __init__(self, name):
        self.name = str(name)
    def getName(self):
        return self.name
    def __str__(self):
        return self.name
    
class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return str(self.src) + '->' + str(self.dest)
    
class WeightedEdge(Edge):
    def __init__(self, src, dest, weight = 1.0):
        self.src = src
        self.dest = dest
        self.weight = weight
    def getWeight(self):
        return self.weight
    def __str__(self):
        return str(self.src)+'->('+str(self.weight)+')'+str(self.dest)

In [24]:
class Digraph(object):
    """edges is a dictionary mapping each node to a list of its children"""
    def __init__(self):
        self.nodes = []
        self.edges = {}
    def addNode(self, node):
        if node in self.nodes:
            raise ValueError('Duplicate node')
        else:
            self.nodes.append(node)
            self.edges[node] = []
            
    def getNode(self,name):
        for node in self.nodes:
            if node.getName() == name:
                return node
            
    def addEdge(self, edge):
        src = edge.getSource()
#         print(src in self.nodes)
        dest = edge.getDestination()
#         print(dest in self.nodes)
        if not(src in self.nodes and dest in self.nodes):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
        
    def childrenOf(self, node):
        return self.edges[node]
    
    def hasNode(self, node):
        return node in self.nodes
    
    def __str__(self):
        res = ''
        for k in self.edges:
            for d in self.edges[k]:
                res += str(k)+'->'+str(d)+'\n'
        return res[:-1]

class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self,edge)
        rev = Edge(edge.getDestination(),edge.getSource())
        Digraph.addEdge(self, rev)

In [26]:
# build the graph
g = Digraph()
cities = ['Boston','Providence','New York','Chicago','Denver','Phoenix','Los Angeles']

for city in cities:
    g.addNode(Node(city))

g.addEdge(Edge(g.getNode('Boston'),g.getNode('Providence')))    
g.addEdge(Edge(g.getNode('Boston'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('Providence'),g.getNode('Boston')))
g.addEdge(Edge(g.getNode('Providence'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('New York'),g.getNode('Chicago')))
g.addEdge(Edge(g.getNode('Chicago'),g.getNode('Denver')))
g.addEdge(Edge(g.getNode('Denver'),g.getNode('Phoenix')))
g.addEdge(Edge(g.getNode('Denver'),g.getNode('New York')))
g.addEdge(Edge(g.getNode('Chicago'),g.getNode('Phoenix')))
g.addEdge(Edge(g.getNode('Los Angeles'),g.getNode('Boston')))
print(g)

Chicago->Denver
Chicago->Phoenix
Denver->Phoenix
Denver->New York
Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
Los Angeles->Boston
New York->Chicago


In [39]:
# exericse: build the graph
# nodes represent permutations of the students in line, edges connect 
# two permutations if one can be made into the other by swapping two adjacent students

def is_permutation(str_a,str_b):
#     print(str_a,str_b)
    for i in range(len(str_a)-1):
        lst_a = list(str_a)
        tmp = lst_a[i]
        lst_a[i] = lst_a[i+1]
        lst_a[i+1] = tmp
        adj_perm = "".join(e for e in lst_a)
#         print(adj_perm)
        if adj_perm == str_b:
#             print("Adjacent Permutation Found")
            return True
#     print("Adjacent Permutation Not Found")
    return False
        
    
# create nodes in the graph
nodes = [Node("ABC"),Node("ACB"),Node("BAC"),Node("BCA"),Node("CAB"),Node("CBA")]
g = Graph()
for n in nodes:
    g.addNode(n)
# create edges in the graph
for i in range(len(nodes)):
    for j in range(i+1,len(nodes)):
        if is_permutation(nodes[i].getName(),nodes[j].getName()):
            g.addEdge(Edge(nodes[i],nodes[j]))
print(g)


CBA->BCA
CBA->CAB
BCA->BAC
BCA->CBA
ACB->ABC
ACB->CAB
ABC->ACB
ABC->BAC
BAC->ABC
BAC->BCA
CAB->ACB
CAB->CBA


### Depth First Search (DFS)

In [109]:
def printPath(path):
    """a path is a list of nodes"""
    result = ''
    for i in range(len(path)):
        if i == len(path) - 1:
            result += str(path[i])
        else:
            result += str(path[i]) + '->'
    return result

def DFS(graph, start, end, path = None, shortest = None):
    """assumes graph is a Digraph, start and end are nodes in graph"""
    if path == None:
        path = []
    path = path + [start]
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: # avoid cycles in graph
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node,end,path,shortest)
                if newPath != None:
                    shortest = newPath
    return shortest

In [110]:
# test DFS
nodes = []
for name in range(10):
    nodes.append(Node(str(name)))
g = Digraph()
for n in nodes:
    g.addNode(n)
    
g.addEdge(Edge(g.getNode('0'),g.getNode('1')))
g.addEdge(Edge(g.getNode('1'),g.getNode('2')))
g.addEdge(Edge(g.getNode('2'),g.getNode('3')))
g.addEdge(Edge(g.getNode('2'),g.getNode('4')))
g.addEdge(Edge(g.getNode('3'),g.getNode('4')))
g.addEdge(Edge(g.getNode('3'),g.getNode('5')))
g.addEdge(Edge(g.getNode('0'),g.getNode('2')))
g.addEdge(Edge(g.getNode('1'),g.getNode('0')))
g.addEdge(Edge(g.getNode('3'),g.getNode('1')))
g.addEdge(Edge(g.getNode('4'),g.getNode('0')))
# g.addEdge(Edge(nodes[0],nodes[1]))
# g.addEdge(Edge(nodes[1],nodes[2]))
# g.addEdge(Edge(nodes[2],nodes[3]))
# g.addEdge(Edge(nodes[2],nodes[4]))
# g.addEdge(Edge(nodes[3],nodes[4]))
# g.addEdge(Edge(nodes[3],nodes[5]))
# g.addEdge(Edge(nodes[0],nodes[2]))
# g.addEdge(Edge(nodes[1],nodes[0]))
# g.addEdge(Edge(nodes[3],nodes[1]))
# g.addEdge(Edge(nodes[4],nodes[0]))

# print(g)

# shortest = DFS(g, nodes[0], nodes[5])
shortest = DFS(g, g.getNode('0'),g.getNode('5'))
print('Shortest path found by DFS: ', printPath(shortest))

Shortest path found by DFS:  0->2->3->5


### Breadth First Search (BFS)